## New chunking method based on titles

In [4]:
import os
from src.chunkers.HTMLChunkNorris import HTMLChunkNorris

In [10]:
html_cn = HTMLChunkNorris()

## LaDrome
SOURCE_FOLDER = "../../../data/D26/ladrome-2023-12-22-page/"

# FILE = "ladrome-page-13830.json"
# FILE = "ladrome-page-10516.json"
# FILE = "ladrome-page-61504.json"
# FILE = "ladrome-page-504967.json"
# FILE = "ladrome-page-567239.json"
# FILE = "ladrome-page-13718.json"
# FILE = "ladrome-page-10516.json"
# FILE = "ladrome-page-547344.json" # big chunks
# FILE = "ladrome-page-574603.json" # no title, just a list of links
# FILE = "ladrome-page-581681.json" # plein de petits titres
# FILE = "ladrome-page-10787.json"
FILE = "ladrome-page-11020.json"

# INSALyon
SOURCE_FOLDER = "../../data/INSALyon/insalyon-2023-12-13-page/"
SOURCE_FOLDER = "../../../data/INSALyon/insalyon-2023-11-27-formations/"


# FILE = "rejoindreinsalyon-page-500.json"
FILE = "bachelor.json"
# FILE = "rejoindreinsalyon-page-160.json"
# FILE = "rejoindreinsalyon-page-4314.json"
# FILE = "rejoindreinsalyon-page-3908.json"

file = HTMLChunkNorris.read_json_file(os.path.join(SOURCE_FOLDER, FILE))
# print(file)
titles = html_cn.get_toc(file)
print(titles)
#chunks = html_cn.get_chunks(titles, FILE)
# print(chunks)
chunks = html_cn(
    os.path.join(SOURCE_FOLDER, FILE),
    max_title_level_to_use="h3",
    max_chunk_word_length=200,
    link_placement="in_sentence",
    chunk_tokens_exceeded_handling="split"
    )
print("\n======================================\n")
for c in chunks:
    print(c["text"])

[{'id': -1, 'text': '', 'level': -1, 'start_position': -1, 'end_position': -1, 'children': [], 'parents': [], 'content': '>'}]


>



### Chunk entire folder

In [2]:
from src.chunkers.HTMLChunkNorris import HTMLChunkNorris

INPUT_FOLDER = "../../../data\INSALyon\insalyon-2023-11-27-formations"
OUTPUT_FOLDER = f"{INPUT_FOLDER}-chunked-v1"

html_cn = HTMLChunkNorris()
html_cn.chunk_entire_directory(
    INPUT_FOLDER,
    OUTPUT_FOLDER,
    max_title_level_to_use="h3",
    max_chunk_word_length=250,
    link_placement="in_sentence",
    chunk_tokens_exceeded_handling="split"
    )

ValueError: Output directory already contains data !